<a href="https://colab.research.google.com/github/Nmn99/Advance-Regression/blob/main/Syntactic_Processing_Assign_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Pre-processing

In [ ]:
# prompt: split data sentence at blank cell

from google.colab import drive
drive.mount('/content/drive')

with open('/content/drive/MyDrive/train_sent', 'r') as train_sent_file:
  train_sentences = train_sent_file.readlines()

with open('/content/drive/MyDrive/train_label', 'r') as train_labels_file:
  train_labels = train_labels_file.readlines()

with open('/content/drive/MyDrive/test_sent', 'r') as test_sent_file:
  test_sentences = test_sent_file.readlines()

with open('/content/drive/MyDrive/test_label', 'r') as test_labels_file:
  test_labels = test_labels_file.readlines()

# Split the sentences at blank lines
train_sentences_split = []
current_sentence = ""
for sentence in train_sentences:
  if sentence.strip() == "":
    if current_sentence:
      train_sentences_split.append(current_sentence.strip())
      current_sentence = ""
  else:
    current_sentence += sentence
if current_sentence:
  train_sentences_split.append(current_sentence.strip())

# You can apply the same logic to other lists (train_labels, test_sentences, test_labels)
# Example for test_sentences:
test_sentences_split = []
current_sentence = ""
for sentence in test_sentences:
  if sentence.strip() == "":
    if current_sentence:
      test_sentences_split.append(current_sentence.strip())
      current_sentence = ""
  else:
    current_sentence += sentence
if current_sentence:
    test_sentences_split.append(current_sentence.strip())

# Now train_sentences_split and test_sentences_split contain sentences split at blank lines
# You can print or further process these lists.
print(len(train_sentences_split))
print(len(test_sentences_split))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
2599
1056


In [ ]:
# prompt: split label at the same length as sentences

train_labels_split = []
current_label = ""
for label in train_labels:
  if label.strip() == "":
    if current_label:
      train_labels_split.append(current_label.strip())
      current_label = ""
  else:
    current_label += label
if current_label:
  train_labels_split.append(current_label.strip())

test_labels_split = []
current_label = ""
for label in test_labels:
  if label.strip() == "":
    if current_label:
      test_labels_split.append(current_label.strip())
      current_label = ""
  else:
    current_label += label
if current_label:
  test_labels_split.append(current_label.strip())

print(len(train_labels_split))
print(len(test_labels_split))

#Check if the lengths of sentences and labels are equal
if len(train_sentences_split) != len(train_labels_split):
  print("Warning: Number of train sentences and labels do not match.")
if len(test_sentences_split) != len(test_labels_split):
  print("Warning: Number of test sentences and labels do not match.")

2599
1056


In [ ]:
# prompt: print sentence and label 10

print(train_sentences_split[10])
train_labels_split[10]


A
2-
,
3-
,
and
4-day
mean
temperature
prior
to
the
test
date
was
compared
to
AFI
using
a
Spearman-rank
Correlation


'O\nO\nO\nO\nO\nO\nO\nO\nO\nO\nO\nO\nO\nO\nO\nO\nO\nO\nO\nO\nO\nO'

# Concept identification

In [ ]:
# prompt: We use POS Taggging to identify the top 25 nouns and pronouns present in the data

import nltk
from collections import Counter

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

def identify_top_nouns_pronouns(sentences):
    """
    Identifies the top 25 nouns and pronouns in a list of sentences.

    Args:
    sentences: A list of strings, where each string is a sentence.

    Returns:
    A list of tuples, where each tuple contains a word and its count,
    sorted in descending order of count.
    """

    nouns_pronouns = []
    for sentence in sentences:
        tokens = nltk.word_tokenize(sentence)
        tagged_tokens = nltk.pos_tag(tokens)
        for word, tag in tagged_tokens:
            if tag.startswith('NN') or tag.startswith('PRP'):  # Noun or pronoun
                nouns_pronouns.append(word.lower()) # Lowercase for consistent counting

    word_counts = Counter(nouns_pronouns)
    top_25 = word_counts.most_common(25)
    return top_25


top_nouns_pronouns_train = identify_top_nouns_pronouns(train_sentences_split)
print("Top 25 nouns and pronouns in training data:")
print(top_nouns_pronouns_train)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Top 25 nouns and pronouns in training data:
[('patients', 366), ('treatment', 209), ('%', 161), ('cancer', 144), ('therapy', 122), ('study', 121), ('we', 108), ('disease', 105), ('lung', 86), ('results', 84), ('cell', 74), ('group', 71), ('effects', 71), ('use', 65), ('chemotherapy', 63), ('>', 60), ('gene', 57), ('effect', 57), ('to_see', 55), ('health', 54), ('women', 51), ('surgery', 51), ('patient', 50), ('children', 49), ('risk', 48)]


# Defining the features for CRF

In [ ]:
# prompt:  define a function getFeaturesForOneWord(sentence, pos):
# Word-Level Features:
# Word Itself: The actual word, often lowercased.
# Word Shape: Information about the word's capitalization (e.g., all uppercase, all lowercase, title case, mixed case).
# Word Length: The length of the word.
# Prefix and Suffix: The first few and last few characters of the word.
# Part-of-Speech (POS) Tag: The grammatical role of the word (e.g., noun, verb, adjective).
# Contextual Features:
# Previous Word: The previous word in the sentence.
# Next Word: The next word in the sentence.
# Previous Tag: The NER tag of the previous word.
# Next Tag: The NER tag of the next word.
# Sentence Position: The position of the word within the sentence (e.g., beginning, middle, end).

def getFeaturesForOneWord(sentence, pos):
    """
    Extracts features for a single word in a sentence.

    Args:
        sentence: The sentence as a list of words.
        pos: The position of the word in the sentence.

    Returns:
        A dictionary of features.
    """

    word = sentence[pos]
    features = {}

    # Word-Level Features
    features['word'] = word.lower()
    features['word_shape'] = get_word_shape(word)
    features['word_length'] = len(word)
    features['prefix'] = word[:3] if len(word) > 3 else word
    features['suffix'] = word[-3:] if len(word) > 3 else word

    # Assuming you have POS tags available (you'd need to compute these)
    # Replace with your actual POS tagging mechanism
    try:
      features['pos_tag'] = nltk.pos_tag([word])[0][1]
    except:
      features['pos_tag'] = 'UNK'

    # Contextual Features
    if pos > 0:
        features['prev_word'] = sentence[pos - 1].lower()
    else:
        features['prev_word'] = "<START>"

    if pos < len(sentence) - 1:
        features['next_word'] = sentence[pos + 1].lower()
    else:
        features['next_word'] = "<END>"

    # Placeholder for previous and next tags (replace with actual tags)
    features['prev_tag'] = "<START>"  # Replace with actual previous NER tag
    features['next_tag'] = "<END>"  # Replace with actual next NER tag

    features['sentence_position'] = pos / len(sentence)  # Normalized position

    return features

def get_word_shape(word):
  if word.isupper():
    return 'all_caps'
  elif word.islower():
    return 'all_lower'
  elif word.istitle():
    return 'title'
  else:
    return 'mixed'

# Getting the features and the labels of sentences

In [ ]:
# prompt: # Define a function to get features for a sentence
# # using the 'getFeaturesForOneWord' function.
# def getFeaturesForOneSentence(sentence):
#   sentence_list = sentence.split()
#   return [getFeaturesForOneWord(sentence_list, pos) for pos in range(len(sentence_list))]

# Define a function to get features for a sentence
# using the 'getFeaturesForOneWord' function.
def getFeaturesForOneSentence(sentence):
  sentence_list = sentence.split()
  return [getFeaturesForOneWord(sentence_list, pos) for pos in range(len(sentence_list))]

In [ ]:
# prompt: # Define a function to get the labels for a sentence.
# def getLabelsInListForOneSentence(labels):
#   return labels.split()

# Define a function to get the labels for a sentence.
def getLabelsForOneSentence(labels):
  return labels.split()

In [ ]:
# prompt: # Apply function 'getFeaturesForOneSentence' to get features on a single sentence which is at index value 5 in train_sentences
# example_sentence = train_sentences[5]
# print(example_sentence)
# features = getFeaturesForOneSentence(example_sentence)
# features[2]

example_sentence = train_sentences_split[5]
print(example_sentence)
features = getFeaturesForOneSentence(example_sentence)
features[2]

Cesarean
rates
at
tertiary
care
hospitals
should
be
compared
with
rates
at
community
hospitals
only
after
correcting
for
dissimilar
patient
groups
or
gestational
age


{'word': 'at',
 'word_shape': 'all_lower',
 'word_length': 2,
 'prefix': 'at',
 'suffix': 'at',
 'pos_tag': 'IN',
 'prev_word': 'rates',
 'next_word': 'tertiary',
 'prev_tag': '<START>',
 'next_tag': '<END>',
 'sentence_position': 0.08333333333333333}

# Defining input and target variables

In [ ]:
# prompt: X_train = [getFeaturesForOneSentence(sentence) for sentence in train_sentences]
# Y_train = [getLabelsInListForOneSentence(labels) for labels in train_labels]
# X_test = [getFeaturesForOneSentence(sentence) for sentence in test_sentences]
# Y_test = [getLabelsInListForOneSentence(labels) for labels in test_labels]

def getLabelsInListForOneSentence(labels):
  return labels.split()

X_train = [getFeaturesForOneSentence(sentence) for sentence in train_sentences_split]
Y_train = [getLabelsInListForOneSentence(labels) for labels in train_labels_split]
X_test = [getFeaturesForOneSentence(sentence) for sentence in test_sentences_split]
Y_test = [getLabelsInListForOneSentence(labels) for labels in test_labels_split]

In [ ]:
!pip install sklearn-crfsuite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.9 MB/s eta 0:00:00


In [ ]:
import sklearn_crfsuite

from sklearn_crfsuite import metrics

# Building the model

In [ ]:
# prompt: # Build the CRF model.
# crf = sklearn_crfsuite.CRF(max_iterations=100)
# crf.fit(X_train, Y_train)

# Build the CRF model.
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, Y_train)

CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    max_iterations=100)

# Evaluation

In [ ]:
# prompt: # Calculate the f1 score using the test data
# Y_pred = crf.predict(X_test)
# metrics.flat_f1_score(Y_test, Y_pred, average='weighted')

# Calculate the f1 score using the test data
Y_pred = crf.predict(X_test)
f1_score = metrics.flat_f1_score(Y_test, Y_pred, average='weighted')
print(f"F1 score: {f1_score}")

F1 score: 0.9273980063381014


# Identifying the diseases and predicted treatment using a custom NER

In [ ]:
# prompt: map all 'D' to T'
# logic behind mapping is they are in the same sentence
# use y_pred to get values of 'D' and 'T'

# Assuming Y_pred is already calculated as in the provided code.
# Create a list to store the modified predictions.
Y_pred_mapped = []

for sentence_pred in Y_pred:
  mapped_sentence = []
  for i, label in enumerate(sentence_pred):
    if label == 'D':
      # Check if the next label in the sequence is 'T'.
      if i + 1 < len(sentence_pred) and sentence_pred[i+1] == 'T':
          mapped_sentence.append('T')  # Map 'D' to 'T'
      else:
          mapped_sentence.append(label)  # Keep 'D' if no subsequent 'T'
    else:
      mapped_sentence.append(label)
  Y_pred_mapped.append(mapped_sentence)


#Now Y_pred_mapped contains the predictions with 'D' mapped to 'T' where applicable.
#You can use Y_pred_mapped for further analysis or evaluation.
#Example: calculate the f1-score with the mapped predictions
f1_score_mapped = metrics.flat_f1_score(Y_test, Y_pred_mapped, average='weighted')
print(f"F1 score (after mapping): {f1_score_mapped}")

Mapping of 'D' to 'T':
'epilepsy': 'Methylphenidate'
'myocardial': 'Gene'
'infarction': 'assist'
'unstable': 'roxithromycin'
'coronary-artery': 'Antichlamydial'
'disease': 'inhibitor'
'myofascial': 'ENS'
'trigger': 'EMS'
'Obesity': 'dexfenfluramine'
'primary': 'transpupillary'
'essential': 'moxonidine'
'foot': 'G-CSF'
'infection': 'omeprazole'
'stroke': 'atrial'
'intracranial': 'double-bolus'
'hemorrhage': 'method'
'rheumatoid': 'arthrodesis'
'early': 'Ropinirole'
'Parkinson': 'tolcapone'
'sore': 'Antibiotics'
'postoperative': 'Homeopathy'
'pain': 'antiinflammatory'
'relief': '('
'abdominal': 'steroids'
'Female': 'surgical'
'Stress': 'procedures'
'stress': 'surgery'
'preeclampsia': 'insemination'
'major': 'thrombolytic'
'malignant': 'alfa-2a'
'pleural': ','
'mesothelioma': 'radiotherapy'
'hearing': 'ECMO'
'pulmonary': 'lung'
'non-obstructive': 'testicular'
'azoospermia': 'sperm'
'testicular': 'fine'
'bleeding': 'needle'
'extratunical': 'aspiration'
'acute': 'Neostigmine'
'migraine': '5

In [ ]:
# prompt: map 'T' when 'D' = retinoblastoma
# use y_pred to map "T"
# logic behind mapping is map same sentence value

# Assuming 'Y_pred' contains the predicted labels and 'test_sentences_split' contains the sentences.
# The logic for mapping 'T' when 'D' = retinoblastoma is improved below

# Updated mapping logic to handle cases where 'D' and 'T' might not be adjacent.
# The mapping is now based on the sentence index and word position, prioritizing 'T' within the same sentence.

def map_t_to_d(y_pred, sentences):
    d_to_t_mapping = {}
    for i, (sentence, predicted_labels) in enumerate(zip(sentences, y_pred)):
      sentence_tokens = sentence.split()
      for j, (word, label) in enumerate(zip(sentence_tokens, predicted_labels)):
          if label == "D" and word.lower() == "retinoblastoma":
            # Check for a 'T' label in the same sentence.
            for k, (t_word, t_label) in enumerate(zip(sentence_tokens, predicted_labels)):
              if t_label == "T":
                  d_to_t_mapping[i] = t_word # Store the mapped value using sentence index
                  break  # Stop searching once a 'T' is found
            break  # Stop searching if a 'D' is found and processed

    return d_to_t_mapping

# Call the function to get the mapping
d_to_t_mapping = map_t_to_d(Y_pred, test_sentences_split)

# Print the mapping
print("Mapping of sentence index to the word labeled as 'T' when 'D' is retinoblastoma:")
for sentence_index, t_value in d_to_t_mapping.items():
    print(f"Sentence {sentence_index + 1}: '{t_value}'")

Mapping of sentence index to the word labeled as 'T' when 'D' is retinoblastoma:
Sentence 47: 'radiotherapy'
